# Load Data

In [16]:
import numpy as np 
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/walmart-sales-analysis/Walmart_sales_Dashboard.pbix
/kaggle/input/walmart-sales-analysis/Walmart Sales Analysis And Marketing Recommendations.pdf
/kaggle/input/walmart-sales-analysis/Walmart_sales_analysis.csv


In [17]:
PATH = '/kaggle/input/walmart-sales-analysis/Walmart_sales_analysis.csv'

df = pd.read_csv(PATH)
df.head(5)

,Store_Number,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,2/5/2010,"1,643,691",0,42.31,2.572,211,8.106
1,1,2/12/2010,"1,641,957",1,38.51,2.548,211,8.106
2,1,2/19/2010,"1,611,968",0,39.93,2.514,211,8.106
3,1,2/26/2010,"1,409,728",0,46.63,2.561,211,8.106
4,1,3/5/2010,"1,554,807",0,46.50,2.625,211,8.106


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6435 entries, 0 to 6434
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store_Number  6435 non-null   int64  
 1   Date          6435 non-null   object 
 2   Weekly_Sales  6435 non-null   object 
 3   Holiday_Flag  6435 non-null   int64  
 4   Temperature   6435 non-null   float64
 5   Fuel_Price    6435 non-null   float64
 6    CPI          6435 non-null   int64  
 7   Unemployment  6435 non-null   float64
dtypes: float64(3), int64(3), object(2)
memory usage: 402.3+ KB


In [19]:
df.describe()

,Store_Number,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,6435.000000,6435.000000,6435.000000,6435.000000,6435.000000,6435.000000
mean,23.000000,0.069930,60.663782,3.358607,171.583994,7.999151
std,12.988182,0.255049,18.444933,0.459020,39.351305,1.875885
min,1.000000,0.000000,-2.060000,2.472000,126.000000,3.879000
25%,12.000000,0.000000,47.460000,2.933000,132.000000,6.891000
50%,23.000000,0.000000,62.670000,3.445000,183.000000,7.874000
75%,34.000000,0.000000,74.940000,3.735000,213.000000,8.622000
max,45.000000,1.000000,100.140000,4.468000,227.000000,14.313000


# Cleaning Data

In [20]:
df.columns.tolist()

['Store_Number',
 'Date',
 'Weekly_Sales',
 'Holiday_Flag',
 'Temperature',
 'Fuel_Price',
 ' CPI ',
 'Unemployment']

In [21]:
df.rename(columns={' CPI ': 'CPI'}, inplace=True)
df.columns.tolist()

['Store_Number',
 'Date',
 'Weekly_Sales',
 'Holiday_Flag',
 'Temperature',
 'Fuel_Price',
 'CPI',
 'Unemployment']

In [22]:
df['Weekly_Sales'] = df['Weekly_Sales'].apply(lambda x: x.replace(',','')).astype(int)
df['Date'] = pd.to_datetime(df['Date'])
df['Store_Number'] = df['Store_Number'].astype(str)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6435 entries, 0 to 6434
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Store_Number  6435 non-null   object        
 1   Date          6435 non-null   datetime64[ns]
 2   Weekly_Sales  6435 non-null   int64         
 3   Holiday_Flag  6435 non-null   int64         
 4   Temperature   6435 non-null   float64       
 5   Fuel_Price    6435 non-null   float64       
 6   CPI           6435 non-null   int64         
 7   Unemployment  6435 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(3), object(1)
memory usage: 402.3+ KB


In [24]:
# Ekstraksi waktu
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Week'] = df['Date'].dt.isocalendar().week
df['Day'] = df['Date'].dt.day_name()

# Cek ulang
df.head()

,Store_Number,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Week,Day
0,1,2010-02-05,1643691,0,42.31,2.572,211,8.106,2010,2,5,Friday
1,1,2010-02-12,1641957,1,38.51,2.548,211,8.106,2010,2,6,Friday
2,1,2010-02-19,1611968,0,39.93,2.514,211,8.106,2010,2,7,Friday
3,1,2010-02-26,1409728,0,46.63,2.561,211,8.106,2010,2,8,Friday
4,1,2010-03-05,1554807,0,46.50,2.625,211,8.106,2010,3,9,Friday


# EDA: Exploratory Data Analysis

## 📈 Tren Penjualan Mingguan

Grafik ini membantu melihat bagaimana penjualan berubah dari minggu ke minggu. Kita bisa mengidentifikasi:
- Tren tahunan (naik/turun?)
- Musiman (misalnya naik di akhir tahun?)
- Outlier atau lonjakan besar (misalnya karena libur besar)

In [25]:
# Agregasi penjualan per minggu
weekly_sales = df.groupby('Date')['Weekly_Sales'].sum().reset_index()
avg_weekly_sales =  weekly_sales['Weekly_Sales'].mean()
median_weekly_sales =  weekly_sales['Weekly_Sales'].median()
print(f'Rata-rata penjualan perminggu: {avg_weekly_sales:.2f}')
print(f'Median penjualan perminggu: {median_weekly_sales:.2f}')
# Plotly line chart
fig = px.line(
    weekly_sales,
    x='Date',
    y='Weekly_Sales',
    title='📈 Tren Penjualan Mingguan Walmart (Seluruh Toko)',
    labels={'Weekly_Sales': 'Penjualan Mingguan', 'Date': 'Tanggal'},
    template='plotly_white'
)

# Menambahkan garis vertikal untuk menunjukkan mean weekly sales.
fig.add_hline(
    y = avg_weekly_sales,
    line_dash="dash",
    line_color="green",
    annotation_text=f"Rata-rata: {avg_weekly_sales:.2f}",
    annotation_position="top left"
)

# Menambahkan garis vertikal untuk menunjukkan median weekly sales.
fig.add_hline(
    y=median_weekly_sales,
    line_dash="dot",
    line_color="red",
    annotation_text=f"Median: {median_weekly_sales:.2f}",
    annotation_position="top right"
)


fig.update_layout(
    title_x=0.5,
    hovermode='x unified',
    xaxis_title='Tanggal',
    yaxis_title='Penjualan (USD)'
)

fig.show()

Rata-rata penjualan perminggu: 47113419.64
Median penjualan perminggu: 46243902.00


* *💡 Apakah ada lonjakan besar penjualan?*

Ada lonjakan besar jauh lebih besar dari mean dan median saat memasuki akhir tahun. Lonjakan terjadi dimulai akhir november seperti tanggal 26 November 2010 penjualan perminggu mencapai sekitar 65.82 kemudian sempat turun ke angka sekitar 49.91 di tanggal 3 Desember 2010 kemudian mencapai puncaknya pada tanggal 24 Desember 2010. Menyambut hari natal, penjualan mengalami kenaikan sebanyak ~71% lebih tinggi daripada rata-rata penjualan mingguan. Pola ini muncul konsisten pada akhir 2011 dan 2012, menandakan **musiman tahunan yang kuat** menjelang Natal dan tahun baru. 

💡 Insight Bisnis:
> Walmart mencatat lonjakan tahunan menjelang Natal. Ini merupakan momentum penting untuk strategi diskon, pengadaan stok, dan penguatan logistik — terutama di bulan November hingga minggu keempat Desember.

## 🎉 Pengaruh Hari Libur terhadap Penjualan

Kita akan membandingkan rata-rata penjualan mingguan antara minggu libur (`Holiday_Flag = 1`) dan minggu biasa (`Holiday_Flag = 0`) untuk mengetahui apakah hari libur berdampak signifikan pada penjualan Walmart.

Kita juga akan mencari tahu minggu libur mana yang paling berdampak terhadap kenaikan penjualan.

In [26]:
# Buat kolom label untuk grafik
df['Holiday_Label'] = df['Holiday_Flag'].map({0: 'Bukan Hari Libur', 1: 'Hari Libur Besar'})

# Agregasi rata-rata penjualan
holiday_avg = df.groupby('Holiday_Label')['Weekly_Sales'].mean().reset_index()


fig = px.bar(
    holiday_avg,
    x='Holiday_Label',
    y='Weekly_Sales',
    color='Holiday_Label',
    text='Weekly_Sales',
    title='🎉 Rata-rata Penjualan Mingguan: Libur vs Non-Libur',
    labels={'Weekly_Sales': 'Rata-rata Penjualan Mingguan'},
    template='plotly_white'
)

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_x=0.5, showlegend=False)
fig.show()

*💡 Apakah minggu libur menghasilkan rata-rata penjualan yang lebih tinggi daripada minggu biasa?*

Tampaknya hari libur besar memiliki rata-rata penjualan yang lebih tinggi daripada bukan hari libur. Namun peningkatan tidak besar secara presentase. Bukan hari libur memiliki rata-rata penjualan sebesar ~1.0 M sedangkan Hari libur besar memiliki rata-rata penjualan sebesar ~1.1M atau hanya naik ~1.0%.

🧠 **Interpretasi:**
- Walaupun secara visual terlihat adanya kenaikan, secara **proporsional** dampaknya tidak terlalu besar.
- Hal ini menunjukkan bahwa **tidak semua minggu libur secara otomatis mendongkrak penjualan secara ekstrem**.
- Kenaikan yang paling signifikan kemungkinan terjadi hanya pada **libur besar tertentu**, seperti **Natal atau Thanksgiving**, bukan semua hari libur.

💡 **Rekomendasi Bisnis:**
> Walmart dapat mempertimbangkan untuk lebih fokus pada minggu libur strategis yang terbukti menghasilkan lonjakan besar (misal, menjelang akhir tahun), daripada menyamaratakan seluruh libur dalam kampanye marketing.

In [27]:
from scipy.stats import ttest_ind

# Pisahkan penjualan minggu libur dan non-libur
sales_holiday = df[df['Holiday_Flag'] == 1]['Weekly_Sales']
sales_non_holiday = df[df['Holiday_Flag'] == 0]['Weekly_Sales']

# Uji T dua sampel
t_stat, p_value = ttest_ind(sales_holiday, sales_non_holiday, equal_var=False)

print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")

# Interpretasi
if p_value < 0.05:
    print("✅ Perbedaan penjualan antara minggu libur dan non-libur SIGNIFIKAN.")
else:
    print("❌ Tidak ada perbedaan signifikan secara statistik.")

T-statistic: 2.6801
P-value: 0.0076
✅ Perbedaan penjualan antara minggu libur dan non-libur SIGNIFIKAN.


In [28]:
holiday_weeks = df[df['Holiday_Flag'] == 1].groupby('Date')['Weekly_Sales'].sum().reset_index()

fig = px.line(
    holiday_weeks,
    x='Date',
    y='Weekly_Sales',
    title='📅 Tren Penjualan Minggu Libur Walmart',
    labels={'Weekly_Sales': 'Penjualan Mingguan'},
    template='plotly_white'
)
fig.update_layout(title_x=0.5)
fig.show()

*🧠 Insight: Dampak Hari Libur Terhadap Penjualan*

Meskipun hasil uji T menunjukkan perbedaan signifikan secara statistik (p = 0.0076), visualisasi tren mengungkapkan bahwa:

- **Lonjakan besar hanya terjadi di beberapa minggu libur spesifik** — kemungkinan besar Natal dan Thanksgiving.
- Sebagian besar minggu libur lain tidak menunjukkan peningkatan penjualan yang signifikan.
- Terjadi **pola konsumsi terpusat**, di mana penjualan melonjak sebelum libur besar, lalu menurun tajam sesudahnya.

💡 Kesimpulan:
> Walmart perlu menyesuaikan strategi kampanye hari libur. Fokus pada **libur besar berdampak tinggi (Q4)** akan jauh lebih efisien daripada menyebar effort ke semua minggu libur.

## 📉 Analisis: Apakah Penjualan Minggu Libur Dipengaruhi Suhu dan Tingkat Pengangguran?

In [29]:
fig = px.scatter(
    df[df['Holiday_Flag'] == 1],
    x='Temperature',
    y='Weekly_Sales',
    color='Year',
    hover_data=['Date'],
    trendline='ols',
    title='🌡️ Suhu vs Penjualan Minggu Libur',
    labels={'Temperature': 'Suhu (°F)', 'Weekly_Sales': 'Penjualan'}
)
fig.update_layout(title_x=0.5)
fig.show()

**1️⃣ Suhu vs Penjualan**

Distribusi antara suhu (°F) dan penjualan juga menunjukkan:

- **Tidak ada tren naik/turun yang signifikan secara linear.**
- Penjualan tinggi terjadi baik saat suhu dingin (20–40°F) maupun sedang (60–70°F).
- Artinya, suhu tidak menjadi faktor penentu utama dalam minggu libur.

💡 *Interpretasi:* 
> Suhu tidak berdampak langsung terhadap penjualan mingguan Walmart selama minggu libur. Hal ini mungkin karena belanja dilakukan **dalam ruangan (indoor retail)** dan tidak terlalu dipengaruhi oleh kondisi cuaca.

In [30]:
fig = px.scatter(
    df[df['Holiday_Flag'] == 1],
    x='Unemployment',
    y='Weekly_Sales',
    color='Year',
    title='💼 Pengangguran vs Penjualan (Minggu Libur)',
    hover_data=['Date'],
    trendline='ols',
    labels={'Unemployment': 'Tingkat Pengangguran (%)', 'Weekly_Sales': 'Penjualan'}
)
fig.update_layout(title_x=0.5)
fig.show()

**2️⃣ Tingkat Pengangguran vs Penjualan**

Scatterplot menunjukkan distribusi penjualan mingguan selama minggu libur terhadap tingkat pengangguran. Secara umum:

- Tidak terlihat korelasi linear yang kuat.
- Meskipun regresi linear menunjukkan tren penurunan kecil, nilai penyebaran data sangat besar.
- Beberapa minggu libur tetap mencatat penjualan tinggi bahkan saat pengangguran tinggi.

💡 *Interpretasi:* 
> Dalam konteks minggu libur, faktor eksternal seperti pengangguran tampaknya **tidak terlalu menghambat konsumsi**. Hal ini bisa dijelaskan oleh perilaku konsumen yang tetap berbelanja untuk liburan sebagai kebutuhan emosional/sosial, meskipun dalam tekanan ekonomi.

## 📌 Kesimpulan:

Faktor seperti **pengangguran dan suhu** tampaknya **tidak memengaruhi penjualan minggu libur secara linear**. Hal ini menguatkan argumen bahwa:
- **Momentum libur besar itu unik**, didorong oleh faktor psikologis, tradisi, dan diskon/promosi besar.
- Strategi promosi Walmart untuk minggu libur **tidak perlu terlalu disesuaikan dengan variabel ekonomi jangka pendek**.

# ✅ Ringkasan Insight: Walmart Weekly Sales Analysis

## 1️⃣ Tren Penjualan Mingguan
- Penjualan mingguan cenderung stabil di kisaran **$1 juta–$1.1 juta**, namun terdapat lonjakan tajam menjelang akhir tahun.
- **Puncak tertinggi** terjadi pada minggu sebelum Natal, dengan kenaikan hingga **+71%** dari rata-rata.
- Terdapat pola musiman yang kuat setiap akhir tahun.

## 2️⃣ Dampak Hari Libur
- Rata-rata penjualan minggu libur lebih tinggi dibanding minggu biasa (~$1.1M vs ~$1.0M).
- Uji statistik **t-test** menunjukkan perbedaan ini **signifikan secara statistik** (p = 0.0076).
- Namun, tidak semua minggu libur berdampak besar — hanya beberapa minggu seperti **Thanksgiving dan Natal** yang benar-benar memberikan lonjakan besar.

## 3️⃣ Variabel Eksternal (Suhu & Ekonomi)
- **Tingkat Pengangguran** tidak memiliki korelasi linear yang kuat dengan penjualan pada minggu libur. Bahkan saat pengangguran tinggi, penjualan libur tetap tinggi.
- **Suhu** juga tidak menunjukkan hubungan langsung. Ini mendukung asumsi bahwa belanja retail Walmart bersifat indoor dan relatif tahan terhadap cuaca.
- Insight ini menekankan bahwa **perilaku konsumen saat minggu libur** lebih dipengaruhi oleh faktor psikologis, tradisi, dan diskon daripada ekonomi jangka pendek.

---

## 🎯 Rekomendasi Bisnis

- Fokus kampanye promosi pada minggu **Thanksgiving dan Natal**, karena terbukti mendorong penjualan signifikan.
- Tidak semua liburan layak diperlakukan setara — **alokasi budget promosi sebaiknya disesuaikan dampak historisnya**.
- Walmart tidak perlu terlalu khawatir terhadap variabel ekonomi musiman (seperti suhu atau pengangguran) dalam merancang strategi minggu libur.
- Untuk tahun-tahun ke depan, pertimbangkan penguatan logistik dan stok di akhir November hingga akhir Desember untuk memaksimalkan potensi penjualan.